In [98]:
import sframe
import string
import pandas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn import metrics
def remove_punctuation(text):
    return text.translate(None, string.punctuation)

products = sframe.SFrame('amazon_baby.gl/')
products = products.fillna('review',{'review':''})  # fill in N/A's in the review column
products = products[products['rating'] != 3]
products['review_clean'] = products['review'].apply(remove_punctuation) #creates a new column
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

#print products
#products.print_rows(num_rows=1)
#products.print_rows(num_rows=1,num_columns=4)
#print products[products['sentiment']==-1]


train_data, test_data = products.random_split(.8, seed=1)

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping

test_matrix = vectorizer.transform(test_data['review_clean'])
#print train_matrix.shape
log_reg=linear_model.LogisticRegression()

log_reg.fit(train_matrix,train_data['sentiment'])


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [99]:
count=(log_reg.coef_>=0).sum()
print count

85830


In [100]:

sample_test_data = test_data[10:13]
#print sample_test_data
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = log_reg.decision_function(sample_test_matrix)
print scores


[  5.60106442  -3.15214773 -10.42523329]


In [101]:
test_result=log_reg.predict_proba(sample_test_matrix)
tmp=test_result[:,1]
tmp=list(tmp)
print tmp.index(min(tmp))+1

3


In [114]:
test_result=log_reg.predict_proba(test_matrix)
test_data['prob']=test_result[:,1];
test_sort_data=test_data.sort('prob',ascending=0)
test_sort_data.print_rows(num_rows=20, num_columns=1)

+-------------------------------+-----+
|              name             | ... |
+-------------------------------+-----+
| Baby Einstein Around The W... | ... |
| Fisher-Price Cradle 'N Swi... | ... |
| P'Kolino Silly Soft Seatin... | ... |
| Evenflo X Sport Plus Conve... | ... |
| Baby Jogger City Mini GT S... | ... |
| Graco Pack 'n Play Element... | ... |
| Infantino Wrap and Tie Bab... | ... |
| Diono RadianRXT Convertibl... | ... |
| Evenflo 6 Pack Classic Gla... | ... |
| Roan Rocco Classic Pram St... | ... |
| Simple Wishes Hands-Free B... | ... |
| Freemie Hands-Free Conceal... | ... |
| Mamas &amp; Papas 2014 Urb... | ... |
| Britax 2012 B-Agile Stroll... | ... |
| Buttons Cloth Diaper Cover... | ... |
| Graco FastAction Fold Jogg... | ... |
| Britax Decathlon Convertib... | ... |
| Ikea 36 Pcs Kalas Kids Pla... | ... |
| Summer Infant Wide View Di... | ... |
| Baby Jogger City Mini GT D... | ... |
+-------------------------------+-----+
[33336 rows x 6 columns]



In [115]:
test_sort_data=test_data.sort('prob',ascending=1)
test_sort_data=test_sort_data[test_sort_data['prob']<0.5]
test_sort_data.print_rows(num_rows=20, num_columns=1)

+-------------------------------+-----+
|              name             | ... |
+-------------------------------+-----+
| Fisher-Price Ocean Wonders... | ... |
| Levana Safe N'See Digital ... | ... |
| Safety 1st Exchangeable Ti... | ... |
| Adiri BPA Free Natural Nur... | ... |
| VTech Communications Safe ... | ... |
| The First Years True Choic... | ... |
| Safety 1st High-Def Digita... | ... |
| Cloth Diaper Sprayer--styl... | ... |
| Motorola Digital Video Bab... | ... |
| Philips AVENT Newborn Star... | ... |
| Ellaroo Mei Tai Baby Carri... | ... |
| Cosco Alpha Omega Elite Co... | ... |
| Peg-Perego Tatamia High Ch... | ... |
| Belkin WeMo Wi-Fi Baby Mon... | ... |
| Chicco Cortina KeyFit 30 T... | ... |
| NUK Cook-n-Blend Baby Food... | ... |
| VTech Communications Safe ... | ... |
| Safety 1st Deluxe 4-in-1 B... | ... |
| Thirsties Hemp Inserts 2 P... | ... |
| Regalo My Cot Portable Bed... | ... |
+-------------------------------+-----+
[4597 rows x 6 columns]



In [117]:
test_result=log_reg.predict(test_matrix)
#print test_result
test_data['pred']=test_result
accuracy=((test_data['pred']==test_data['sentiment']).sum())/float(test_data.num_rows())
print round(accuracy,2)

0.93


In [119]:
train_result=log_reg.predict(train_matrix)
#print test_result
train_data['pred']=train_result
accuracy=((train_data['pred']==train_data['sentiment']).sum())/float(train_data.num_rows())
print round(accuracy,2)

0.97


In [121]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [127]:
simple_model=linear_model.LogisticRegression()
simple_model.fit(train_matrix_word_subset,train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [128]:
simple_model_coef_table = sframe.SFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})
#print simple_model_coef_table

In [129]:

simple_model_coef_table=simple_model_coef_table.sort('coefficient',ascending=0)

In [130]:
count=(simple_model_coef_table['coefficient']>0).sum()
print count
print simple_model_coef_table[simple_model_coef_table['coefficient']>0]

10
+-----------------+---------+
|   coefficient   |   word  |
+-----------------+---------+
|  1.67307389259  |  loves  |
|  1.50981247669  | perfect |
|  1.36368975931  |   love  |
|  1.19253827349  |   easy  |
|  0.943999590572 |  great  |
|  0.520185762718 |  little |
|  0.503760457768 |   well  |
|  0.190908572064 |   able  |
|  0.085512779463 |   old   |
| 0.0588546711528 |   car   |
+-----------------+---------+
[? rows x 2 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use len(sf) to force materialization.


In [131]:
train_result=simple_model.predict(train_matrix_word_subset)
train_data['pred']=train_result
accuracy=((train_data['pred']==train_data['sentiment']).sum())/float(train_data.num_rows())
print round(accuracy,2)

0.87


In [133]:
test_result=simple_model.predict(test_matrix_word_subset)
test_data['pred']=test_result
accuracy=((test_data['pred']==test_data['sentiment']).sum())/float(test_data.num_rows())
print round(accuracy,2)

0.87


In [1]:
from sklearn import dummy
dum=dummy.DummyClassifier()
dum.fit(train_matrix_word_subset,train_data['sentiment'])
test_result=dum.predict(test_matrix_word_subset)
test_data['pred']=test_result
accuracy=((test_data['pred']==test_data['sentiment']).sum())/float(test_data.num_rows())
print round(accuracy,2)

NameError: name 'train_matrix_word_subset' is not defined